In [1]:
caffe_root  = '/mnt/lustre/chendagui/text_detect/'
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe
import os
from caffe import layers as L, params as P
from caffe.proto import caffe_pb2

In [2]:
def CNN(lmdb, batch_size):
    n = caffe.NetSpec()
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                         transform_param=dict(scale=1./255), ntop=2)
    # conv11
    n.conv11 = L.Convolution(n.data, kernel_size=3, num_output=32, pad=1, weight_filler=dict(type='xavier'))
    n.relu11 = L.ReLU(n.conv11, in_place=True)
    n.bn11 = L.BatchNorm(n.relu11)
    # conv12
    n.conv12 = L.Convolution(n.bn11, kernel_size=3, num_output=64, stride=2 ,pad=1, weight_filler=dict(type='xavier'))
    n.relu12 = L.ReLU(n.conv12, in_place=True)
    n.bn12 = L.BatchNorm(n.relu12)
    # conv21
    n.conv21 = L.Convolution(n.bn12, kernel_size=3, num_output=128, pad=1, weight_filler=dict(type='xavier'))
    n.relu21 = L.ReLU(n.conv21, in_place=True)
    n.bn21 = L.BatchNorm(n.relu21)
    # conv22
    n.conv22 = L.Convolution(n.bn21, kernel_size=3, num_output=128, stride=2, pad=1, weight_filler=dict(type='xavier'))
    n.relu22 = L.ReLU(n.conv22, in_place=True)
    n.bn22 = L.BatchNorm(n.relu22)
    
    #out
    n.pool = L.Pooling(n.bn22, pool=P.Pooling.AVE, global_pooling=True)
    n.score = L.InnerProduct(n.pool, num_output=10, weight_filler=dict(type='xavier'))
    n.loss = L.SoftmaxWithLoss(n.score, n.label)
    n.acc = L.Accuracy(n.score, n.label, include=dict(phase=caffe.TEST))
    
    return n.to_proto()

In [3]:
train_path = 'CNN_train.prototxt'
test_path = 'CNN_test.prototxt'
solver_path = 'CNN_solver.prototxt'
with open(train_path, 'w') as f:
    f.write(str(CNN('/mnt/lustre/chendagui/deform_mnist/deform_train_lmdb/', 64)))
    
with open(test_path, 'w') as f:
    f.write(str(CNN('/mnt/lustre/chendagui/deform_mnist/deform_test_lmdb/', 100)))

In [10]:
def solver(train_path, test_path, snapshot):
    s = caffe_pb2.SolverParameter()
    s.random_seed = 0xCAFFE
    s.train_net = train_path
    s.test_net.append(test_path)
    s.test_interval = 1000
    s.test_iter.append(1000)
    s.max_iter = 20000
    s.type = "SGD"
    s.base_lr = 0.001
    s.momentum = 0.9
    s.weight_decay = 5e-4
    s.lr_policy = 'inv'
    s.gamma = 0.0001
    s.power = 0.75
    s.display = 1000
    s.snapshot = 5000
    s.snapshot_prefix = snapshot
    s.solver_mode = caffe_pb2.SolverParameter.GPU
    return s

In [11]:
with open(solver_path, 'w') as f:
    f.write(str(solver(train_path, test_path, 'snapshot/CNN')))

In [6]:
def deform_CNN(lmdb, batch_size, trainable=True):
    if trainable:
        param = [dict(lr_mult=1), dict(lr_mult=1)]
        global_stats = False
    else:
        param = [dict(lr_mult=0), dict(lr_mult=0)]
        global_stats = True
    n = caffe.NetSpec()
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                         transform_param=dict(scale=1./255), ntop=2)
    # conv11
    n.conv11 = L.Convolution(n.data, kernel_size=3, num_output=32, pad=1, param=param, weight_filler=dict(type='xavier'))
    n.relu11 = L.ReLU(n.conv11, in_place=True)
    n.bn11 = L.BatchNorm(n.relu11, use_global_stats=global_stats)
    n.offset11 = L.Convolution(n.bn11, kernel_size=3, num_output=2, pad=1, weight_filler=dict(type='constant', value=0))
    n.deform11 = L.DeformConv(n.bn11, n.offset11, kernel_h=1, kernel_w=1, propagate_down=[1,1])
    # conv12
    
    n.conv12 = L.Convolution(n.deform11, kernel_size=3, num_output=64, stride=2 ,pad=1, param=param, weight_filler=dict(type='xavier'))
    n.relu12 = L.ReLU(n.conv12, in_place=True)
    n.bn12 = L.BatchNorm(n.relu12, use_global_stats=global_stats)
    n.offset12 = L.Convolution(n.bn12, kernel_size=3, num_output=2, pad=1, weight_filler=dict(type='constant', value=0))
    n.deform12 = L.DeformConv(n.bn12, n.offset12, kernel_h=1, kernel_w=1, propagate_down=[1,1])
    
    # conv21
    n.conv21 = L.Convolution(n.deform12, kernel_size=3, num_output=128, pad=1, param=param, weight_filler=dict(type='xavier'))
    n.relu21 = L.ReLU(n.conv21, in_place=True)
    n.bn21 = L.BatchNorm(n.relu21, use_global_stats=global_stats)
    n.offset21 = L.Convolution(n.bn21, kernel_size=3, num_output=2, pad=1, weight_filler=dict(type='constant', value=0))
    n.deform21 = L.DeformConv(n.bn21, n.offset21, kernel_h=1, kernel_w=1, propagate_down=[1,1])
    
    # conv22
    n.conv22 = L.Convolution(n.deform21, kernel_size=3, num_output=128, stride=2, pad=1, param=param, weight_filler=dict(type='xavier'))
    n.relu22 = L.ReLU(n.conv22, in_place=True)
    n.bn22 = L.BatchNorm(n.relu22, use_global_stats=global_stats)
    
    #out
    n.pool = L.Pooling(n.bn22, pool=P.Pooling.AVE, global_pooling=True)
    n.score = L.InnerProduct(n.pool, num_output=10,  param=param, weight_filler=dict(type='xavier'))
    n.loss = L.SoftmaxWithLoss(n.score, n.label)
    n.acc = L.Accuracy(n.score, n.label, include=dict(phase=caffe.TEST))
    
    return n.to_proto()

In [7]:
deform_train_path = 'deform_train.pt'
deform_test_path = 'deform_test.pt'
deform_solver_path = 'deform_solver.pt'

In [8]:
with open(deform_train_path, 'w') as f:
    f.write(str(deform_CNN('/mnt/lustre/chendagui/deform_mnist/deform_train_lmdb/', 64, True)))
with open(deform_test_path, 'w') as f:
    f.write(str(deform_CNN('/mnt/lustre/chendagui/deform_mnist/deform_test_lmdb/', 100, True)))

In [12]:
with open(deform_solver_path, 'w') as f:
    f.write(str(solver(deform_train_path, deform_test_path, 'snapshot/deform_CNN')))